In [64]:
S3_INPUT_PATH = "s3://wri-projects/Aqueduct30/test/testGpd/"
EC2_INPUT_PATH = "/volumes/data/temp/"
EC2_OUTPUT_PATH = "/volumes/data/temp/output/"
S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/test/output/"

In [59]:
!mkdir -p {EC2_INPUT_PATH}
!mkdir -p {EC2_OUTPUT_PATH}

In [31]:
!aws s3 cp {S3_INPUT_PATH} {EC2_INPUT_PATH} --recursive

download: s3://wri-projects/Aqueduct30/test/testGpd/FAO/faoBuffered.dbf to ../../../../data/temp/FAO/faoBuffered.dbf
download: s3://wri-projects/Aqueduct30/test/testGpd/FAO/faoBuffered.cpg to ../../../../data/temp/FAO/faoBuffered.cpg
download: s3://wri-projects/Aqueduct30/test/testGpd/Hybas/hybas_lev06_v1c_merged_fiona_V01.prj to ../../../../data/temp/Hybas/hybas_lev06_v1c_merged_fiona_V01.prj
download: s3://wri-projects/Aqueduct30/test/testGpd/Hybas/hybas_lev06_v1c_merged_fiona_V01.cpg to ../../../../data/temp/Hybas/hybas_lev06_v1c_merged_fiona_V01.cpg
download: s3://wri-projects/Aqueduct30/test/testGpd/Hybas/hybas_lev06_v1c_merged_fiona_Cropped_V01.qpj to ../../../../data/temp/Hybas/hybas_lev06_v1c_merged_fiona_Cropped_V01.qpj
download: s3://wri-projects/Aqueduct30/test/testGpd/FAO/faoBuffered.qpj to ../../../../data/temp/FAO/faoBuffered.qpj
download: s3://wri-projects/Aqueduct30/test/testGpd/Hybas/hybas_lev06_v1c_merged_fiona_Cropped_V01.cpg to ../../../../data/temp/Hybas/hybas_lev0

In [14]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib notebook
import os
import folium

In [3]:
from shapely.wkt import loads
from shapely.geometry import Point

In [49]:
gdfFAO = gpd.read_file('/volumes/data/temp/FAO/faoBuffered.shp')
gdfHybas = gpd.read_file('/volumes/data/temp/Hybas/hybas_lev06_v1c_merged_fiona_Cropped_V01.shp')

In [51]:
gdfHybas.head()

,COAST,DIST_MAIN,DIST_SINK,ENDO,HYBAS_ID,MAIN_BAS,NEXT_DOWN,NEXT_SINK,ORDER,PFAF_ID,SORT,SUB_AREA,UP_AREA,geometry
0,0,227.1,227.1,0,2060499090,2060021030,2060502710,2060021030,2,232260,526,13664.2,13664.2,"POLYGON ((1.133333333333358 47.35833333333336,..."
1,0,227.3,227.3,0,2060498990,2060021030,2060502710,2060021030,1,232270,527,10041.2,42572.4,"POLYGON ((2.87916666666669 46.73750000000003, ..."
2,0,275.6,275.6,0,2060455290,2060022150,2060446440,2060022150,1,232405,542,1088.5,44526.2,"POLYGON ((2.079166666666683 48.98750000000003,..."
3,0,275.7,275.7,0,2060455180,2060022150,2060446440,2060022150,2,232404,543,16783.0,16783.0,"POLYGON ((3.770833333333363 49.19166666666669,..."
4,0,353.6,353.6,0,2060459800,2060022150,2060455290,2060022150,1,232407,544,7203.8,12783.1,"POLYGON ((2.90416666666669 49.13750000000003, ..."


In [66]:
gdfHybas.set_index('PFAF_ID')

,COAST,DIST_MAIN,DIST_SINK,ENDO,HYBAS_ID,MAIN_BAS,NEXT_DOWN,NEXT_SINK,ORDER,SORT,SUB_AREA,UP_AREA,geometry
PFAF_ID,,,,,,,,,,,,,
232260,0,227.1,227.1,0,2060499090,2060021030,2060502710,2060021030,2,526,13664.2,13664.2,"POLYGON ((1.133333333333358 47.35833333333336,..."
232270,0,227.3,227.3,0,2060498990,2060021030,2060502710,2060021030,1,527,10041.2,42572.4,"POLYGON ((2.87916666666669 46.73750000000003, ..."
232405,0,275.6,275.6,0,2060455290,2060022150,2060446440,2060022150,1,542,1088.5,44526.2,"POLYGON ((2.079166666666683 48.98750000000003,..."
232404,0,275.7,275.7,0,2060455180,2060022150,2060446440,2060022150,2,543,16783.0,16783.0,"POLYGON ((3.770833333333363 49.19166666666669,..."
232407,0,353.6,353.6,0,2060459800,2060022150,2060455290,2060022150,1,544,7203.8,12783.1,"POLYGON ((2.90416666666669 49.13750000000003, ..."
232406,0,353.8,353.8,0,2060459960,2060022150,2060455290,2060022150,2,545,30654.0,30654.8,"POLYGON ((4.554166666666687 47.20416666666669,..."
232408,0,613.4,613.4,0,2060461380,2060022150,2060459800,2060022150,2,546,2134.4,2134.5,"POLYGON ((5.200000000000026 48.56666666666669,..."
232409,0,613.6,613.6,0,2060461420,2060022150,2060459800,2060022150,1,547,3444.4,3444.5,"POLYGON ((5.475000000000028 48.01666666666669,..."
232502,0,0.0,0.0,0,2060022380,2060022380,0,2060022380,1,549,6194.5,6194.8,"POLYGON ((3.512500000000028 49.92500000000003,..."


In [73]:
gdfHybasBuffer = gdfHybas['geometry'].buffer(-0.005,resolution=16)

In [70]:
gdfHybas[]

0    POLYGON ((0.4802056493726044 47.33676526482385...
1    POLYGON ((0.3946555531847661 47.49782138220293...
2    POLYGON ((1.985697993274555 48.88870001490769,...
3    POLYGON ((1.642712429100236 49.6413141345426, ...
4    POLYGON ((2.414370437159644 48.81750000000003,...
dtype: object

In [60]:
gdfHybasBuffer.to_file(os.path.join(EC2_OUTPUT_PATH,'output.shp'))

In [65]:
!aws s3 cp {EC2_OUTPUT_PATH} {S3_OUTPUT_PATH} --recursive

upload: ../../../../data/temp/output/output.cpg to s3://wri-projects/Aqueduct30/test/output/output.cpg
upload: ../../../../data/temp/output/output.shx to s3://wri-projects/Aqueduct30/test/output/output.shx
upload: ../../../../data/temp/output/output.prj to s3://wri-projects/Aqueduct30/test/output/output.prj
upload: ../../../../data/temp/output/output.shp to s3://wri-projects/Aqueduct30/test/output/output.shp
upload: ../../../../data/temp/output/output.dbf to s3://wri-projects/Aqueduct30/test/output/output.dbf
